<a href="https://colab.research.google.com/github/eutiagovski/projetos-cursos/blob/main/datascience-mentorama/12_Arvores_Ensembles_exercicio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Modelos baseados em árvores

<br>

Ao longo do módulo, discutimos bastante as árvores de decisão, bem como ensemble de árvores, como Random Forest e algoritmos do tipo boosting.

Esses __ensembles acabam tendo muitos hiperparâmetros;__ escolhe-los de forma manual acaba sendo muito custoso e tedioso. 

Neste exercício, vamos discutir a respeito da metolodia __grid-search__, que otimiza essa busca de hiperparâmetros.

Considere o dataset abaixo (basta executar as células):

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [21]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

In [22]:
#problema de regressão

X, y = load_diabetes().data, load_diabetes().target
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.25, random_state = 42)
print(Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape)

(331, 10) (111, 10) (331,) (111,)


Imagine que queremos testar - usando cross-validation - várias instâncias de Random Forests: com 10 árvores, com 100 árvores, com 1000 árvores, com profundidade máxima 1, 5, 10. 

Como podemos proceder? O código abaixo exemplifica um jeito:

In [23]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

for n_est in [10,100,100]:
    for prof in [1,5,10]:
        rf = RandomForestRegressor(n_estimators=n_est, max_depth=prof)
        cvres = cross_val_score(estimator=rf, X = Xtrain, y = ytrain, cv = 3, scoring='r2')
        print("estimators: ", n_est, " prof: ", prof, " | R2 mean / std: ", cvres.mean(), ' / ', cvres.std())

estimators:  10  prof:  1  | R2 mean / std:  0.3355481330416495  /  0.040349941345889206
estimators:  10  prof:  5  | R2 mean / std:  0.41989776651558436  /  0.06280878775898815
estimators:  10  prof:  10  | R2 mean / std:  0.3964704372108536  /  0.03389580978621192
estimators:  100  prof:  1  | R2 mean / std:  0.33267819044403446  /  0.03494111033741746
estimators:  100  prof:  5  | R2 mean / std:  0.43310775428686094  /  0.046999779699282596
estimators:  100  prof:  10  | R2 mean / std:  0.42157375976689887  /  0.036353504781212784
estimators:  100  prof:  1  | R2 mean / std:  0.33059773480951454  /  0.03955872355594707
estimators:  100  prof:  5  | R2 mean / std:  0.4158624516163554  /  0.05151959439289268
estimators:  100  prof:  10  | R2 mean / std:  0.41999230151383465  /  0.03657050497935747


Podemos, com algum trabalho, escolher o melhor modelo.

Se quisermos testar mais parâmetros, podemos aumentar nosso loop... mais isso vai ficando cada vez mais complicado.

A proposta do __grid-search__ é justamente fazer isso de forma mais automática!

Podemos importar a função GridSearchCV do módulo model_selection do sklearn e usá-la para isso. 
Na prática, precisamos definir um __estimador base__ para o grid. Além disso, precisamos definir um __dicionário de parâmetros__ a ser testado. Ainda, definiremos a quantidade de folds para cross-validation e qual a métrica de performance que queremos otimizar:

In [24]:
#importando a função
from sklearn.model_selection import GridSearchCV

In [25]:
#definindo o estimador base
estimador_base = RandomForestRegressor()

#definindo o dicionario de parâmetros do modelo
params_RF = {"n_estimators":[10,1000], "max_depth":[2,10]}

In [26]:
grid = GridSearchCV(estimator = estimador_base, 
                    param_grid = params_RF, 
                    scoring = 'r2', 
                    cv = 3)

grid

GridSearchCV(cv=3, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 10], 'n_estimators': [10, 1000]},
             scoring='r2')

In [27]:
#treinando os modelos no grid
grid.fit(Xtrain, ytrain)

GridSearchCV(cv=3, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 10], 'n_estimators': [10, 1000]},
             scoring='r2')

O objeto "grid", após o treinamento acima, conterá várias informações muito relevantes. 

__1- "best_params_":__ retorna os melhores parâmetros, de acordo com a métrica de performance avaliada na cross-validation;

__1- "best_score_":__ retorna o melhor score - métrica de performance - nos dados de validação;

__1- "best_estimator_":__ retorna o melhor modelo, já treinado;

__1- "cv_results_":__ retorna uma visão geral dos resultados.

In [28]:
grid.best_params_

{'max_depth': 10, 'n_estimators': 1000}

In [29]:
grid.best_score_

0.4214073288035971

In [30]:
grid.best_estimator_

RandomForestRegressor(max_depth=10, n_estimators=1000)

In [31]:
grid.cv_results_

{'mean_fit_time': array([0.01508888, 1.32203134, 0.01897724, 1.86235547]),
 'mean_score_time': array([0.00156816, 0.08508396, 0.00172544, 0.08978383]),
 'mean_test_score': array([0.38550617, 0.4078309 , 0.37735677, 0.42140733]),
 'param_max_depth': masked_array(data=[2, 2, 10, 10],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[10, 1000, 10, 1000],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 2, 'n_estimators': 10},
  {'max_depth': 2, 'n_estimators': 1000},
  {'max_depth': 10, 'n_estimators': 10},
  {'max_depth': 10, 'n_estimators': 1000}],
 'rank_test_score': array([3, 2, 4, 1], dtype=int32),
 'split0_test_score': array([0.29500008, 0.34795264, 0.27252484, 0.3677299 ]),
 'split1_test_score': array([0.42292768, 0.44584111, 0.44681913, 0.45204133]),
 'split2_test_score': array([0.43859073, 0.42969897, 0.41272634

__Exercício 1:__ Utilizando o dataset abaixo, faça um grid_search com KNN's, Random Forests e GradientBoostings e retorne o melhor modelo de cada tipo.

__Obs.:__ Lembre-se de fazer um pré-processamento nos dados!

In [34]:
#preco_mediano_das_casas é a variável target
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/preco_casas.csv")
print(df.shape)
df.head()

(20640, 10)


,longitude,latitude,idade_mediana_das_casas,total_comodos,total_quartos,populacao,familias,salario_mediano,preco_mediano_das_casas,proximidade_ao_mar
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,PERTO DA BAÍA
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,PERTO DA BAÍA
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,PERTO DA BAÍA
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,PERTO DA BAÍA
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,PERTO DA BAÍA


In [66]:
## Preparando os dados utilizando a função já criada em aulas anteriores:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

dftrain, dftest = train_test_split(df)

def preprocessamento_completo(df, dataset_de_treino = True, cat_encoder = None, std_scaler = None):

    dff = df.copy()

    #criação de novas variáveis
    dff["comodos_por_familia"] = dff["total_comodos"]/dff["familias"]
    dff["quartos_por_comodos"] = dff["total_quartos"]/dff["total_comodos"]
    dff["populacao_por_familia"]= dff["populacao"]/dff["familias"]

    #retirando valores faltantes
    dff = dff.dropna(axis = 0)
    
    variaveis_para_normalizar = ['latitude',
                                 'longitude',
                                'idade_mediana_das_casas',
                                 'total_comodos',
                                 'total_quartos',
                                 'populacao',
                                 'familias',
                                 'salario_mediano',
                                 'comodos_por_familia',
                                 'quartos_por_comodos',
                                 'populacao_por_familia']

    if dataset_de_treino:  
        
        #OHE
        encoder = OneHotEncoder()
        df_prox_mar_OHE = encoder.fit_transform(dff[['proximidade_ao_mar']]).toarray()

        #normalização
        sc = StandardScaler()
        variaveis_norm = sc.fit_transform(dff[variaveis_para_normalizar])
        
        X, y =  np.c_[df_prox_mar_OHE, variaveis_norm], dff.preco_mediano_das_casas.values
        return X, y, encoder, sc
    
    else:
        #OHE
        df_prox_mar_OHE = cat_encoder.transform(dff[['proximidade_ao_mar']]).toarray()
        
        #normalização
        variaveis_norm = std_scaler.transform(dff[variaveis_para_normalizar]) 
        
        X, y =  np.c_[df_prox_mar_OHE, variaveis_norm], dff.preco_mediano_das_casas.values
        return X, y

In [68]:
Xtrain, ytrain, encoder_train, scaler_train  = preprocessamento_completo(df = dftrain, dataset_de_treino = True, cat_encoder = None, std_scaler = None)
Xtrain.shape, ytrain.shape, dftrain.shape

In [341]:
Xtest, ytest = preprocessamento_completo(df = dftest, dataset_de_treino = False, cat_encoder = encoder_train, std_scaler = scaler_train)
Xtest.shape, ytest.shape, dftest.shape

((5160, 16), (5160,), (5160, 10))

In [123]:
def search_best(estimator, params, X, y):
  grid = GridSearchCV(estimator = estimator, 
                      param_grid = params, 
                      scoring = 'r2', 
                      cv = 3)

  grid.fit(X, y)
  return grid.best_params_, grid.best_score_


In [126]:
# Knn

from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor()

knn_params = [{"n_neighbors":[1,5,10,100]}]
knn_best, knn_score = search_best(knn, knn_params, Xtrain, ytrain)

knn_best, knn_score

({'n_neighbors': 10}, 0.7176486623455928)

In [130]:
# Forest

from sklearn.ensemble import RandomForestRegressor

fr = RandomForestRegressor()

fr_params = [{'n_estimators': [3,10,30], 'max_features': [2, 4, 6, 8], "max_depth":[2,10]}]
fr_best, fr_score = search_best(fr, fr_params, Xtrain, ytrain)
fr_best, fr_score


({'max_depth': 10, 'max_features': 8, 'n_estimators': 30}, 0.7938555679803637)

In [131]:
# Gradient Boost

from sklearn.ensemble import GradientBoostingRegressor

gb = GradientBoostingRegressor()

gb_params = [{'n_estimators': [3,10,30], 'learning_rate': [0.001, 0.1, 0.25, 1, 2], "max_depth":[2,10]}]

gb_best, gb_score = search_best(gb, gb_params, Xtrain, ytrain)
gb_best, gb_score

({'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 30},
 0.811556047278731)

__Exercício 2:__ Crie uma classe para comparar o grid_search dentre vários modelos distintos.
    
    
Essa classe, gridSearchAll(), já está pré-desenvolvida no código abaixo. O exercício consiste de __completar essa classe.__ Para isso, crie o métodos fit_all, que irá treinar, usando grid_search, todos os grids que tenham sido pré-construídos e inseridos na classe.
Ainda, a quantidade de folds para a validação cruzada no grid_search deve ser implementada no método construtor da classe, bem como qual a métrica de performance a ser avaliada. 
Finalmente, salve o melhor modelo de cada grid e tenha um método best_all_grid_models que retorna o melhor modelo dentre todos os grids.

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [314]:
class gridSearchAll():
    
    def __init__(self):
        self.grid_models = []
        self.scores = []
        #num_folds...
    
    def insert_model(self, estimator_base, param_grid):
      self.grid_models.append([estimator_base, param_grid])
        
    def fit_all(self, X, y, scoring='r2', folds=3):
      scores = []

      for est in range(len(self.grid_models)):
        grid = GridSearchCV(estimator = self.grid_models[est][0], param_grid = self.grid_models[est][1], scoring = scoring, cv = folds)
        grid.fit(X, y)

        print(f'Model: {self.grid_models[est][0]}, best_params: {grid.best_params_}, score: {grid.best_score_}')
        scores.append([grid.estimator, grid.best_params_, grid.best_score_])
      
      self.grid = grid
      self.scores = scores
      print('Model Trained')

    def best_all_grid_models(self):
      dataframe = pd.DataFrame(self.scores, columns=['Model', 'Best Params', 'Score'])
      best = dataframe[dataframe.Score == dataframe.Score.max()].values[0]

      self.best = best
      print(f'Best model: {best[0]}. Best parameters: {best[1]}. Score: {best[2]} ({self.grid.scorer_})')
      

In [315]:
gd = gridSearchAll()

In [316]:
gd.grid_models

[]

In [317]:
params_RF

{'max_depth': [2, 10], 'n_estimators': [10, 1000]}

In [318]:
gd.insert_model(estimator_base = RandomForestRegressor(), param_grid = params_RF)

In [319]:
gd.grid_models

[[RandomForestRegressor(), {'max_depth': [2, 10], 'n_estimators': [10, 1000]}]]

In [320]:
from sklearn.neighbors import KNeighborsRegressor

In [321]:
gd.insert_model(estimator_base = KNeighborsRegressor(), param_grid = {"n_neighbors":[1,2,10]})

In [322]:
gd.grid_models

[[RandomForestRegressor(), {'max_depth': [2, 10], 'n_estimators': [10, 1000]}],
 [KNeighborsRegressor(), {'n_neighbors': [1, 2, 10]}]]

In [323]:
from sklearn.ensemble import GradientBoostingRegressor

params_gb = {'n_estimators': [3, 10, 30], 'learning_rate': [0.001, 0.1, 0.25, 1, 2, 5], "max_depth":[2,10]}


In [324]:
params_gb = {'n_estimators': [1, 3, 10, 100], 'learning_rate': [0.01, 0.25, 1, 1.25, 5]}

gd.insert_model(estimator_base=GradientBoostingRegressor(), param_grid=params_gb)

In [325]:
gd.grid_models

[[RandomForestRegressor(), {'max_depth': [2, 10], 'n_estimators': [10, 1000]}],
 [KNeighborsRegressor(), {'n_neighbors': [1, 2, 10]}],
 [GradientBoostingRegressor(),
  {'learning_rate': [0.01, 0.25, 1, 1.25, 5],
   'n_estimators': [1, 3, 10, 100]}]]

__Exercício 3:__ Usando a classe criada, analise novamente os modelos criados no exercício 1.

In [326]:
gd.fit_all(Xtrain, ytrain)

Model: RandomForestRegressor(), best_params: {'max_depth': 10, 'n_estimators': 1000}, score: 0.7909453251999164
Model: KNeighborsRegressor(), best_params: {'n_neighbors': 10}, score: 0.7176486623455928
Model: GradientBoostingRegressor(), best_params: {'learning_rate': 0.25, 'n_estimators': 100}, score: 0.8136696606328382
Model Trained


In [327]:
gd.best_all_grid_models()

Best model: GradientBoostingRegressor(). Best parameters: {'learning_rate': 0.25, 'n_estimators': 100}. Score: 0.8136696606328382 (make_scorer(r2_score))


In [332]:
gb = GradientBoostingRegressor(learning_rate=0.25, n_estimators=100)
gb.fit(Xtrain, ytrain)

GradientBoostingRegressor(learning_rate=0.25)

In [365]:
from sklearn.metrics import r2_score

In [366]:
r2_score(ytest, gb.predict(Xtest))

0.8056798932976598